In [211]:
import pandas as pd
import numpy as np
import random
import math

In [212]:
df = pd.read_csv("mock_dataset_cleaned.csv")
df.head(10) # All time in str, need convert for binning

,customer,interarrival_time,service_time,arrival_time,service_start,service_end,wait_time,process_duration,queue_length,rating,price_paid,postal_code,lat,long
0,1,4.78,10.30,10.07,10.07,10.25,0.00,10.30,1,3,10,508913,1.37093,103.97119
1,2,3.83,7.22,10.13,10.25,10.37,6.47,13.68,2,5,10,540330,1.39681,103.88936
2,3,9.78,6.77,10.30,10.37,10.48,3.92,10.67,2,3,10,670260,1.38840,103.78793
3,4,0.38,10.42,11.30,11.30,11.48,0.00,10.42,1,4,8,460211,1.32560,103.93241
4,5,0.58,13.18,11.32,11.48,11.70,9.83,23.03,2,1,10,470631,1.33331,103.91448
5,6,6.05,9.82,11.42,11.70,11.87,16.97,26.78,3,1,10,330027,1.31884,103.86282
6,7,17.00,13.45,11.70,11.87,12.08,9.78,23.22,2,3,10,570510,1.34878,103.84970
7,8,3.03,6.63,11.75,12.08,12.20,20.18,26.82,3,2,10,650156,1.34836,103.74384
8,9,1.13,9.28,11.77,12.20,12.35,25.68,34.98,4,2,10,680623,1.39832,103.74734
9,10,18.62,10.73,12.08,12.35,12.53,16.37,27.10,4,2,10,121441,1.31483,103.76460


In [213]:
df.describe()

,customer,interarrival_time,service_time,arrival_time,service_start,service_end,wait_time,process_duration,queue_length,rating,price_paid,postal_code,lat,long
count,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000
mean,5139.500000,13.592008,9.980245,11.992642,12.058929,12.050701,0.766316,10.767438,0.716287,4.177856,11.140397,554744.458358,1.364155,103.854687
std,2967.147367,12.414917,2.003323,6.941320,6.950136,6.951646,2.346301,3.090067,0.875465,0.790308,3.987628,150596.036673,0.030839,0.079605
min,1.000000,0.000000,2.530000,0.000000,0.000000,0.000000,0.000000,2.480000,0.000000,1.000000,8.000000,121441.000000,1.314830,103.719350
25%,2570.250000,4.100000,8.630000,5.880000,5.980000,6.000000,0.000000,8.870000,0.000000,4.000000,10.000000,510440.000000,1.348360,103.773790
50%,5139.500000,9.720000,9.980000,11.970000,12.120000,12.130000,0.000000,10.350000,1.000000,4.000000,10.000000,530804.000000,1.357210,103.871970
75%,7708.750000,19.450000,11.370000,18.000000,18.050000,18.050000,0.000000,12.015000,1.000000,5.000000,10.000000,650156.000000,1.388400,103.918510
max,10278.000000,59.980000,17.950000,23.980000,23.980000,23.980000,32.170000,39.670000,7.000000,5.000000,25.000000,823273.000000,1.430830,103.971190


In [214]:
#important ones are wait_time, process_duration, queue_length, rating

def calc_bias(row):
    total_bias = 0
    
    wait_time_bias = -row['wait_time'] * 0.02
    
    process_duration_bias = row['process_duration'] * 0.001
    
    queue_length_bias = -0.3 if row['queue_length'] >= 5 else -0.2 if row['queue_length'] >= 3 else 0
    
    rating_bias = -0.4 if row['rating'] <= 2 else 0 if row['rating'] <= 3 else 0.2
    
    total_bias = wait_time_bias + process_duration_bias + queue_length_bias + rating_bias
    
    return total_bias

In [215]:
df['bias'] = df.apply(calc_bias, axis = 1)
df.head()

,customer,interarrival_time,service_time,arrival_time,service_start,service_end,wait_time,process_duration,queue_length,rating,price_paid,postal_code,lat,long,bias
0,1,4.78,10.30,10.07,10.07,10.25,0.00,10.30,1,3,10,508913,1.37093,103.97119,0.01030
1,2,3.83,7.22,10.13,10.25,10.37,6.47,13.68,2,5,10,540330,1.39681,103.88936,0.08428
2,3,9.78,6.77,10.30,10.37,10.48,3.92,10.67,2,3,10,670260,1.38840,103.78793,-0.06773
3,4,0.38,10.42,11.30,11.30,11.48,0.00,10.42,1,4,8,460211,1.32560,103.93241,0.21042
4,5,0.58,13.18,11.32,11.48,11.70,9.83,23.03,2,1,10,470631,1.33331,103.91448,-0.57357


In [216]:
labels = []

for idx, row in df.iterrows():
    
    if 0.65 + row['bias'] > 1:
        a = 1
        
    elif 0.65 + row['bias'] < 0:
        a = 0
        
    else:
        a = 0.65 + row['bias']
    b = 1 - a
    labels.append(np.random.choice([1, 0], p = [a, b]))

df['return'] = labels
df.drop('bias', axis = 1, inplace = True)

In [217]:
df.head()

,customer,interarrival_time,service_time,arrival_time,service_start,service_end,wait_time,process_duration,queue_length,rating,price_paid,postal_code,lat,long,return
0,1,4.78,10.30,10.07,10.07,10.25,0.00,10.30,1,3,10,508913,1.37093,103.97119,0
1,2,3.83,7.22,10.13,10.25,10.37,6.47,13.68,2,5,10,540330,1.39681,103.88936,0
2,3,9.78,6.77,10.30,10.37,10.48,3.92,10.67,2,3,10,670260,1.38840,103.78793,0
3,4,0.38,10.42,11.30,11.30,11.48,0.00,10.42,1,4,8,460211,1.32560,103.93241,1
4,5,0.58,13.18,11.32,11.48,11.70,9.83,23.03,2,1,10,470631,1.33331,103.91448,1


In [218]:
X = pd.DataFrame(df, columns = ['service_start', 'wait_time', 'process_duration', 'queue_length', 'rating', 'price_paid'])
y = df['return']

In [219]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.feature_selection import RFE

In [220]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2022)

In [221]:
clf = LogisticRegression(random_state = 2022)
clf.fit(X_train, y_train.values.ravel())

LogisticRegression(random_state=2022)

In [222]:
clf.coef_

array([[ 0.00214887, -0.15435878, -0.01430409, -0.05010401,  0.48792781,
         0.00108176]])

In [223]:
y_pred = clf.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred)) #accuracy can be adjusted accordingly

Testing accuracy 0.8195525291828794


In [227]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
# poor recall score for negative class. it is okay since business-wise, it is more costly to have FP than FN

[[  59  345]
 [  26 1626]]
              precision    recall  f1-score   support

           0       0.69      0.15      0.24       404
           1       0.82      0.98      0.90      1652

    accuracy                           0.82      2056
   macro avg       0.76      0.57      0.57      2056
weighted avg       0.80      0.82      0.77      2056



In [225]:
rfe = RFE(clf, n_features_to_select=4)
fit = rfe.fit(X, y.values.ravel())
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

selected = fit.support_

#get the top-4 feature_names to see if simulation did work as intended
feature_names = X.columns
print(feature_names.to_numpy()[selected])

Num Features: 4
Selected Features: [False  True  True  True  True False]
Feature Ranking: [3 1 1 1 1 2]
['wait_time' 'process_duration' 'queue_length' 'rating']
